## Error on the val set

In [ ]:
# reading csv file
from pandas import read_csv
# importing pandas module  
import pandas as pd

import matplotlib.pyplot as plt

df = read_csv('../handson-ml2/prediction_credit.csv', index_col=0)

In [ ]:
df

## Error column

In [ ]:
#Create the error column/define the function to show the errors/use the df.apply to apply the function to all rows
def function_matrix(row):
    if row['SeriousDlqin2yrs'] == row['SeriousDlqin2yrs.1'] and row['SeriousDlqin2yrs'] == 0 :
        val = "TN"
    elif row['SeriousDlqin2yrs'] == row['SeriousDlqin2yrs.1'] and row['SeriousDlqin2yrs'] == 1 :
        val = "TP"
    elif row['SeriousDlqin2yrs'] > row['SeriousDlqin2yrs.1']:
        val = "FN"
    else:
        val = "FP"
    return val

df['Error'] = df.apply(function_matrix, axis=1)


In [ ]:
df

## Confusion matrix

In [ ]:
#Create a function to show the confusion matrix using the error column
tn =0
tp =0
fn =0
fp =0

for value in df['Error']:
    if value == 'TN':
        tn +=1 
    elif value == 'FN':
        fn +=1
    elif value == 'TP':
        tp +=1
    elif value == 'FP':
        fp +=1 

print('TN=',tn)
print('FN=',fn)
print('TP=',tp)
print('FP=',fp)

## Optimisation of classification threshold

### Find the 100 most important errors

In [ ]:
#Create an abs column/Desc Order the column/Create and export the first 100 errors
df['absolute_error'] = (df['1 probability']-df['SeriousDlqin2yrs.1']).abs()
desc_order = df.sort_values(by='absolute_error', ascending=False).head(100)
desc_order.to_csv('100_credit_errors.csv')

In [ ]:
df

### Create gain/lost table to obtain the best threshold

In [ ]:
#Create a function to calculate the gain/lost and create a table
def cost_threshold(threshold):
    prediction_threshold = df['1 probability'] > threshold #if the 1-probability is higher than threshold

    #~ = "not" for tables / & = "and" for tables / | = "or" for tables

    P_pred = prediction_threshold
    N_pred = ~prediction_threshold
    
    #P_real:do not rembourse the credit
    #N_real: do rembourse the credit
    P_real = (df['SeriousDlqin2yrs'] == 1)
    N_real = (df['SeriousDlqin2yrs'] == 0)
    
    #Confusion matrix df.loc to localize the row and apply the function through the lenght to finally have the numbers of Tp etc..
    TP = len(df.loc[P_pred & P_real])
    TN = len(df.loc[N_pred & N_real])
    FP = len(df.loc[P_pred & N_real])
    FN = len(df.loc[N_pred & P_real])

    TP_cost = 0
    FN_cost = -2500
    FP_cost = -500
    TN_cost = 500

    cost = TP * TP_cost
    cost += TN * TN_cost
    cost += FP * FP_cost
    cost += FN * FN_cost

    return cost

#Function to optimize the best threshold/for every 'n' apply threshold values between 0-1000
table = [cost_threshold(n/1000) for n in range(0,1000)]
plt.plot(table)

max_val = max(table)
max_threshold = table.index(max_val)/1000
print(f'The maximum gain is {max_val} with a threshold {max_threshold}')


In [ ]:
threshold = 0.281
print("Threshold =", threshold)

In [ ]:
export_csv = desc_order.to_csv (r'credit_prediction_errors analysis.csv', header=True)